In [1]:
import re
import io
import requests
import pandas as pd
from fuzzywuzzy import fuzz, process
from pac_hunter.hunter import fetch_committee_distributions
from pac_hunter.states import us_state_to_abbrev, abbrev_to_us_state
from pac_hunter.utils import fuzzy_filter, clean_df
from pac_hunter.constants import OPENSECRETS_API_KEY, OPENFEC_API_KEY

c:\Users\Kepler\fraypac\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
raypac_house_2020 = pd.read_csv("raypac-house-2020.csv")
raypac_senate_2020 = pd.read_csv("raypac-senate-2020.csv")
jan6_house = pd.read_csv("jan6-house-deniers.csv")
jan6_senate = pd.read_csv("jan6-senate-deniers.csv")
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/election-deniers/fivethirtyeight_election_deniers.csv"
ballot_deniers = pd.read_csv(url)
ballot_deniers = ballot_deniers.assign(party="r")

In [3]:
raypac_2020 = pd.concat([raypac_house_2020.assign(office="Representative"), raypac_senate_2020.assign(office="Senator")], ignore_index=True)
jan6_congress = pd.concat([jan6_house.assign(office="Representative"), jan6_senate.assign(office="Senator")], ignore_index=True)

In [4]:
raypac_2020_clean = clean_df(raypac_2020)
jan6_congress_clean = clean_df(jan6_congress)
ballot_deniers_clean = clean_df(ballot_deniers)

In [5]:
ballot_deniers_clean['last_name'] = ballot_deniers_clean['candidate'].apply(lambda x: x.split(' ')[-1])
ballot_deniers_clean

,candidate,incumbent,state,office,district,stance,source,url,note,party,name,state_code,last_name
0,katie britt,no,alabama,senator,NaN,fully denied,nbc news,https://twitter.com/vaughnhillyard/status/1528...,NaN,r,katie britt,al,britt
1,jerry carl,yes,alabama,representative,1,fully denied,"congressional roll call, alabama political rep...",https://clerk.house.gov/votes/202111?date=01%2...,NaN,r,jerry carl,al,carl
2,barry moore,yes,alabama,representative,2,fully denied,congressional roll call,https://clerk.house.gov/votes/202111?date=01%2...,NaN,r,barry moore,al,moore
3,mike rogers,yes,alabama,representative,3,fully denied,congressional roll call,https://clerk.house.gov/votes/202111?date=01%2...,NaN,r,mike rogers,al,rogers
4,robert aderholt,yes,alabama,representative,4,fully denied,congressional roll call,https://clerk.house.gov/votes/202111?date=01%2...,NaN,r,robert aderholt,al,aderholt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,eric toney,no,wisconsin,attorney general,NaN,accepted with reservations,email to fivethirtyeight,NaN,NaN,r,eric toney,wi,toney
548,amy lynn loudenbeck,no,wisconsin,secretary of state,NaN,accepted with reservations,nbc,https://www.nbcnews.com/politics/2022-election...,NaN,r,amy lynn loudenbeck,wi,loudenbeck
549,harriet hageman,no,wyoming,representative,at-large,fully denied,casper star-tribune,https://trib.com/news/state-and-regional/hagem...,NaN,r,harriet hageman,wy,hageman
550,mark gordon,yes,wyoming,governor,NaN,no comment,NaN,NaN,NaN,r,mark gordon,wy,gordon


In [6]:
jan6_deniers = jan6_congress_clean.query("vote=='yea'")

In [7]:
raypac_2020_deniers = fuzzy_filter(raypac_2020_clean, jan6_deniers, 'candidate', 'name')

In [8]:
raypac_2020_deniers

,candidate,party,state,total,cycle,office,total_float,name,state_code,matched_denier
1,devin nunes,r,calif,"$20,000",2020,representative,20000.0,devin nunes,ca,nunes
2,robert b aderholt,r,ala,"$10,000",2020,representative,10000.0,robert b aderholt,al,aderholt
4,richard w allen,r,ga,"$10,000",2020,representative,10000.0,richard w allen,ga,allen
5,brian babin,r,texas,"$10,000",2020,representative,10000.0,brian babin,tx,babin
7,jim banks,r,ind,"$10,000",2020,representative,10000.0,jim banks,in,banks
19,tom cole,r,okla,"$10,000",2020,representative,10000.0,tom cole,ok,cole
34,michael guest,r,miss,"$10,000",2020,representative,10000.0,michael guest,ms,guest
35,vicky hartzler,r,mo,"$10,000",2020,representative,10000.0,vicky hartzler,mo,hartzler
48,kevin mccarthy,r,calif,"$10,000",2020,representative,10000.0,kevin mccarthy,ca,mccarthy
54,steven palazzo,r,miss,"$10,000",2020,representative,10000.0,steven palazzo,ms,palazzo


In [9]:
total_donations = raypac_2020_deniers["total_float"].sum()
print(f"Raytheon Technologies via RAYPAC donated ${total_donations:.2f} to members of congress that voted against certifying the 2020 election in the 2019-2020 election cycle")

Raytheon Technologies via RAYPAC donated $235500.00 to members of congress that voted against certifying the 2020 election in the 2019-2020 election cycle


In [10]:
from pac_hunter.constants import OPENSECRETS_API_KEY


In [11]:
opensec_url = "https://www.opensecrets.org/api/"
org_name = "raytheon tech"
payload = dict(
    method="getOrgs",
    apikey = OPENSECRETS_API_KEY,
    org = org_name,
    output="json",
)
res = requests.get(opensec_url, params=payload)
org = res.json()["response"]["organization"]["@attributes"]

In [12]:
payload = dict(
    method="orgSummary",
    apikey=OPENSECRETS_API_KEY,
    id=org["orgid"],
    output="json",
)
res = requests.get(opensec_url, params=payload)

In [13]:
res.json()

{'response': {'organization': {'@attributes': {'cycle': '2022',
    'orgid': 'D000072615',
    'orgname': 'Raytheon Technologies',
    'total': '2370989',
    'indivs': '542194',
    'pacs': '1798000',
    'soft': '30795',
    'tot527': '0',
    'dems': '1172640',
    'repubs': '1151387',
    'lobbying': '5900000',
    'outside': '0',
    'mems_invested': '0',
    'gave_to_pac': '2521',
    'gave_to_party': '284264',
    'gave_to_527': '0',
    'gave_to_cand': '1644263',
    'source': 'www.opensecrets.org/orgs/summary.php?id=D000072615'}}}}

In [14]:
url_openfec = "https://api.open.fec.gov/v1"

In [15]:
path = "/names/committees/"
payload = dict(
    api_key = OPENFEC_API_KEY,
    q = "raytheon technologies"
)
res = requests.get(f"{url_openfec}{path}", params=payload)
committee_id = res.json()["results"][0]["id"]

In [16]:
payload = dict(
    api_key = OPENFEC_API_KEY,
    name = "devin nunes",
)
res = requests.get(f"{url_openfec}/candidates/search/", params=payload)
candidate_id = res.json()['results'][0]['candidate_id']

In [17]:
payload = dict(
    api_key = OPENFEC_API_KEY,
    committee_id = [committee_id],
    recipient_name = jan6_congress_clean['name'][:10].to_numpy(),
    per_page = 100,
)
res = requests.get(f"{url_openfec}/schedules/schedule_b/by_recipient/", params=payload)

In [29]:
df = fetch_committee_distributions(api_key=OPENFEC_API_KEY, committee_name="raytheon technologies", recipient_names=ballot_deniers_clean["last_name"])

Multiple results for committee raytheon technologies, taking the first entry
Dividing recipient names into groups of 50
